In [2]:
import TT
from TT import MACD,RSI,KDJ
import pandas as pd
from datetime import datetime, timedelta, date
import time, os
import argparse
import glob

import sys
sys.path.insert(1, '..')

import plotly
import plotly.express as px
import plotly.graph_objects as go


In [ ]:

def generate_dates(start_date, end_date):
	"""Generate a list of dates between start_date and end_date (inclusive)."""
	start = datetime.strptime(start_date, '%Y-%m-%d') - timedelta(days=1)
	end = datetime.strptime(end_date, '%Y-%m-%d')

	# print(start,end)
	
	date_list = []
	delta = timedelta(days=1)
	while start <= end:
		date_list.append(start.strftime('%Y-%m-%d'))
		start += delta
	return date_list

In [ ]:
def get_previous_date(date_string, period=20, date_format='%Y-%m-%d'):
	"""Given a string date, return the date of given days before."""
	original_date = datetime.strptime(date_string, date_format)
	previous_date = original_date - timedelta(days=period)
	return previous_date.strftime(date_format)

def is_valid_date(date_string, date_format='%Y-%m-%d'):
	"""Check if the given string is a valid date."""
	try:
		datetime.strptime(date_string, date_format)
		return True
	except ValueError:
		return False

In [ ]:
def check_cross(line1, line2, m = 5):
	if len(line1) < m or len(line2) < m:
		raise ValueError(f"Both line1 and line2 should have at least {m} values.")

	for i in range(len(line1) - m + 1):
		subset_1 = line1[i:i+m]
		subset_2 = line2[i:i+m]

		# 查找初始金叉
		if subset_1[0] < subset_2[0] and subset_1[1] > subset_2[1]:
			trend = "gold"
			# 检查是否K始终大于D
			if all(k > d for k, d in zip(subset_1[1:], subset_2[1:])):
				return i+1, trend

		# 查找初始死叉
		elif subset_1[0] > subset_2[0] and subset_1[1] < subset_2[1]:
			trend = "dead"
			# 检查是否K始终小于D
			if all(k < d for k, d in zip(subset_1[1:], subset_2[1:])):
				return i+1, trend

	return None, None


In [ ]:
def check_upper_ma(prices, realtime_prices, period, if_minute = False):
	unique_codes = prices['future_code'].unique()
	newest_prices = realtime_prices[["future_code", "newest_price"]]
	ma_selected_code = []
	for code in unique_codes:
		sub_df = prices[prices['future_code'] == code]
		if if_minute:
			close_prices = sub_df["newest_price"]
		else:
			close_prices = sub_df["close"]
		# print(close_prices)
		ma_value = TT.MA(close_prices, period)[-1]

		temp = newest_prices.loc[newest_prices["future_code"] == code,"newest_price"]
		if len(temp) >0:
			if ma_value < temp.item():
				ma_selected_code.append(code)	

	return ma_selected_code
